In [ ]:
# usual imports
import os
import numpy as np
import matplotlib.pyplot as plt

from rail.core.utils import RAILDIR
#from rail.estimation.algos.bpz_version.utils import RAIL_BPZ_DIR
from rail.pipelines.estimation.estimate_all import EstimatePipeline
from rail.core import common_params

from rail.pipelines.utils.name_factory import NameFactory, DataType, CatalogType, ModelType, PdfType
#from rail.core.stage import RailStage, RailPipeline

import qp
import ceci
#import ceci

In [ ]:
#change to your rail location
RAIL_BPZ_DIR = "/path/to/your/rail/rail_bpz/src/"

In [ ]:
bands = 'grizy'
maglims = [27.66, 27.25, 26.6, 26.24, 25.35]
maglim_dict={}
for band,limx in zip(bands, maglims):
    maglim_dict[f"HSC{band}_cmodel_dered"] = limx

In [ ]:
common_params.set_param_defaults(
    bands=[f'HSC{band}_cmodel_dered' for band in bands],
    err_bands=[f'{band}_cmodel_magerr' for band in bands],
    nondetect_val=np.nan,
    ref_band='HSCi_cmodel_dered',
    redshift_col='specz_redshift',
    mag_limits=maglim_dict,
    zmax=6.0,
)

In [ ]:
pipe = EstimatePipeline()

In [ ]:
namer = NameFactory()

In [ ]:
pipe.estimate_somoclu.config.update(spec_groupname = "", redshift_colname = 'specz_redshift')
pipe.estimate_simplesom.config.update(spec_groupname = "", redshift_colname = 'specz_redshift')


In [ ]:
input_file = 'dered_223501_sz_match_pdr3_dud_NONDET.hdf5'


input_dict = dict(
    model_knn=os.path.join(namer.get_data_dir(DataType.model, ModelType.estimator), "model_knn.pkl"),
    model_simplenn=os.path.join(namer.get_data_dir(DataType.model, ModelType.estimator), "model_simplenn.pkl"),
    model_simplesom=os.path.join(namer.get_data_dir(DataType.model, ModelType.estimator), "model_simplesom.pkl"),
    model_somoclu=os.path.join(namer.get_data_dir(DataType.model, ModelType.estimator), "model_somoclu.pkl"),
    model_fzboost=os.path.join(namer.get_data_dir(DataType.model, ModelType.estimator), "model_FZBoost.hdf5"), #_fzboost
    model_trainz=os.path.join(namer.get_data_dir(DataType.model, ModelType.estimator), "model_trainz.pkl"),        
    input=input_file,
    spec_input = input_file,
)

In [ ]:
pipe_info = pipe.initialize(input_dict, dict(output_dir='.', log_dir='.', resume=True), None)


In [ ]:
pipe.save('tmp_estimate_all.yml')

In [ ]:
import ceci
pr = ceci.Pipeline.read('tmp_estimate_all.yml')

In [ ]:
pr.run()

# display $p(z)$

In [ ]:
infile = './pdf/pz/output_simplenn.hdf5'

qp_ensemble = qp.read(infile)

print(qp_ensemble.npdf)
print(qp_ensemble.ancil)

In [ ]:
plt.figure()
plt.hist(qp_ensemble.ancil['zmode'], bins=40)
plt.show()

## display $n(z)$

In [ ]:
infile = './pdf/nz/output_simplesom.hdf5'

qp_ensemble = qp.read(infile)

print(qp_ensemble.npdf)
print(qp_ensemble.ancil)

In [ ]:
plt.figure()
qp_ensemble_single_nz[0].plot_native(xlim = (0,6))
plt.show()